In [20]:
import os
import io
import pandas as pd
import sys
import numpy as np
import twitter
import json
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [21]:
#client = datastore.Client()
#client = language.LanguageServiceClient()

In [22]:
api = twitter.Api(consumer_key="NOrVFOrKaGpOsD1LHjlDmD9lA",
                  consumer_secret="8Lg3hdsmIbEn9Q1fLTiWbCqHD7vp2OSc21mvTGElnudCr5jM7p",
                  access_token_key="49683260-3LQ6XoNAFi9R7MVaSGU1igwNxJkfxL6aFpUJfe7pG",
                  access_token_secret="zNXNyrewzVYC2kIqQ1GlGQoLIDHmTJtjXpMFZhwKGxYb8")

In [23]:
#https://dev.twitter.com/rest/reference/get/search/tweets
# https://python-twitter.readthedocs.io/en/latest/searching.html
# https://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively
sinceId = None
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
max_id = -1
tweetCount = 0
results = api.GetSearch(raw_query="q=%23apple+%23iphone+%23iphonex&result_type=mixed&since=2017-09-10&count=100&lang=en")

forma = [(a.created_at, 
          a.text,
          a.retweet_count, 
          a.favorite_count,
          a.id, 
          a.user.name, 
          a.user.screen_name,
          a.user.description,
          a.user.id,
          a.user.location, 
          a.user.favourites_count, 
          a.user.followers_count,
          a.user.friends_count, 
          a.user.statuses_count, 
          a.user.created_at, 
          a.user.verified, 
          a.user.time_zone, 
          a.coordinates) for a in results]
print(forma)
#https://dev.twitter.com/overview/api/tweets#obj-coordinates

[('Wed Sep 13 13:01:19 +0000 2017', "I can't decide. What do u think??\n\nhttps://t.co/zSOEO7ZVvK\n\n#AppleEvent #AppleEvent2017 #apple #iPhone8 #iPhoneX… https://t.co/JgeUk16SRM", 223, 412, 907952332746493952, 'Sarantos', 'SarantosMelogia', 'Described by industry insiders as an emotionally powerful vocal style masterfully united with a fusion of classic 80s rock & modern music! #newmusic #80smusic', 1498656096, 'Chicago, IL', 52696, 903231, 29656, 10201, 'Mon Jun 10 16:17:47 +0000 2013', True, 'Central Time (US & Canada)', None), ('Mon Sep 18 07:30:05 +0000 2017', "Home button or no home button 😉 it's still dangerous to use while driving. 📱 #AppleEvent #iPhone #Apple #iPhone8… https://t.co/6PLGEsKzws", 30, 21, 909680911515602945, 'THINK! road safety', 'THINKgovuk', 'Official THINK! road safety channel. We encourage safer behaviour to reduce the number of deaths/injuries on our roads. Twitter policy http://t.co/oDLlotPz8f', 1322025240, 'Great Britain', 511, 22233, 448, 2290, 'Tue Apr 0

In [24]:
#https://github.com/agalea91/nhl_2016_playoffs_twitter/blob/master/nhl_playoffs_2016_twitter_study.ipynb
data = {'created_at': [],'text': [], 
        'retweet_count': [], 'tweet_id':[],
        'user_favorites_count': [], 'tweet_location': [],
        'user_followers_count':[], 'user_time_zone':[],
        'user_id':[], 'user_name':[], 
        "screen_name":[], 'user_verified':[],
        'user_location':[], 'user_description':[], 
        'user_statuses_count': [], 'user_created_at':[],
        'user_friends_count': [], 'favorite_count': []}

for t in forma:
    data['created_at'].append(t[0])
    data['text'].append(t[1])
    data['retweet_count'].append(t[2])
    data['favorite_count'].append(t[3])
    data['tweet_id'].append(t[4])
    data['user_name'].append(t[5])
    data['screen_name'].append(t[6])
    data['user_description'].append(t[7])
    data['user_id'].append(t[8])
    data['user_location'].append(t[9])
    data['user_favorites_count'].append(t[10])
    data['user_followers_count'].append(t[11])
    data['user_friends_count'].append(t[12])
    data['user_statuses_count'].append(t[13])
    data['user_created_at'].append(t[14])
    data['user_verified'].append(t[15])
    data['user_time_zone'].append(t[16])
    data['tweet_location'].append(t[17])

df = pd.DataFrame(data)
df.drop_duplicates(inplace=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df['user_created_at'] = pd.to_datetime(df['user_created_at'])

RT = []
for t in data['text']:
    RT.append(t.split()[0]=='RT')
df['RT'] = RT

df.head()

,created_at,favorite_count,retweet_count,screen_name,text,tweet_id,tweet_location,user_created_at,user_description,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,2017-09-13 13:01:19,412,223,SarantosMelogia,I can't decide. What do u think??\n\nhttps://t...,907952332746493952,None,2013-06-10 16:17:47,Described by industry insiders as an emotional...,52696,903231,29656,1498656096,"Chicago, IL",Sarantos,10201,Central Time (US & Canada),True,False
1,2017-09-18 07:30:05,21,30,THINKgovuk,Home button or no home button 😉 it's still dan...,909680911515602945,None,2013-04-02 08:31:05,Official THINK! road safety channel. We encour...,511,22233,448,1322025240,Great Britain,THINK! road safety,2290,Edinburgh,True,False
2,2017-09-14 15:09:24,15,23,THINKgovuk,Here’s everything you need to know about the n...,908346952382861313,None,2013-04-02 08:31:05,Official THINK! road safety channel. We encour...,511,22233,448,1322025240,Great Britain,THINK! road safety,2290,Edinburgh,True,False
3,2017-09-21 00:21:39,0,1,StartUpRealTime,RT @MoptwoLLC: 9 Things Only iPhone Power User...,910660259173236736,None,2016-03-21 14:15:25,Startup News in real time,528764,28109,88,711919143302336512,,StartUpNews,734775,None,False,True
4,2017-09-21 00:20:38,1,1,MoptwoLLC,9 Things Only iPhone Power Users Take Advantag...,910660003962404864,None,2013-11-01 00:19:26,"Moptwo is Social Media with substance. Post, o...",1706,5480,459,2167468956,"Santa Monica, California",Moptwo,15665,None,False,False


In [25]:
df.describe()

,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count
count,100.000000,100.000000,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000
mean,4.880000,7.170000,9.104774e+17,7184.710000,11439.560000,1133.530000,2.066885e+17,31882.130000
std,41.246311,34.426339,3.527254e+14,53045.771624,90349.493497,3631.479904,3.617712e+17,90396.101923
min,0.000000,0.000000,9.079523e+17,0.000000,0.000000,0.000000,7.948150e+05,8.000000
25%,0.000000,0.000000,9.104627e+17,15.500000,147.000000,119.000000,1.611138e+08,820.750000
50%,0.000000,0.000000,9.105314e+17,142.000000,304.000000,315.000000,2.586413e+09,2227.000000
75%,0.000000,1.000000,9.105846e+17,1080.250000,1668.750000,705.250000,1.766720e+17,15665.000000
max,412.000000,258.000000,9.106603e+17,528764.000000,903231.000000,29656.000000,9.101991e+17,734775.000000


In [26]:
len(np.unique(df.text))

93

In [27]:
df.columns[df.isnull().any()]

Index(['tweet_location', 'user_time_zone'], dtype='object')

In [28]:
#https://cloud.google.com/natural-language/docs/sentiment-tutorial

In [29]:
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = True)
df.to_sql('sawi_tweets_historical', engine, if_exists='append')

2017-09-21 02:45:46,598 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2017-09-21 02:45:46,600 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 02:45:46,607 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2017-09-21 02:45:46,608 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 02:45:46,611 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2017-09-21 02:45:46,612 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 02:45:46,616 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2017-09-21 02:45:46,617 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 02:45:46,620 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2017-09-21 02:45:46,621 INFO sqlalchemy.engine.base.Engine {}
2017-09-21 02:45:46,624 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2017-09-21 02

2017-09-21 02:45:46,741 INFO sqlalchemy.engine.base.Engine COMMIT
